# Argumentation Structuree

**Navigation**: [<- Tweety-5-Abstract-Argumentation](Tweety-5-Abstract-Argumentation.ipynb) | [Index](Tweety-1-Setup.ipynb) | [Tweety-7a-Extended-Frameworks ->](Tweety-7a-Extended-Frameworks.ipynb)

---

## Objectifs pedagogiques

1. Maitriser le framework ASPIC+ et sa conversion vers Dung
2. Decouvrir DeLP (Defeasible Logic Programming)
3. Explorer ABA (Assumption-Based Argumentation)
4. Comprendre l'argumentation deductive en logique propositionnelle
5. Utiliser ASP (Answer Set Programming) avec Clingo

## Prerequis

Executez d'abord [Tweety-1-Setup.ipynb](Tweety-1-Setup.ipynb) pour configurer l'environnement JVM.

## Duree estimee : 35-45 minutes

> **Note**: ASP necessite Clingo installe. Le notebook configure automatiquement le chemin si Clingo est disponible.

In [1]:
# --- Initialisation JVM Tweety + Outils Externes ---
print("--- Verification JVM Tweety + Outils ---")
jvm_ready = False

import jpype
import jpype.imports
import os
import pathlib
import shutil
import platform

# === Configuration COMPLETE des outils externes ===
EXTERNAL_TOOLS = {
    "CLINGO": "",
    "SPASS": "",
    "EPROVER": "",
    "SAT_SOLVER_PYTHON": "",
    "MARCO": "",
}

def get_tool_path(tool_name):
    """Retourne le chemin valide d'un outil ou None."""
    path_str = EXTERNAL_TOOLS.get(tool_name, "")
    if not path_str:
        return None
    if shutil.which(path_str):
        return path_str
    path_obj = pathlib.Path(path_str)
    if path_obj.is_file():
        return str(path_obj.resolve())
    if path_obj.is_dir():
        return str(path_obj.resolve())
    return None

# --- Auto-detection des outils ---
system = platform.system()
exe_suffix = ".exe" if system == "Windows" else ""

# 1. Clingo (ASP solver) - Tweety attend le REPERTOIRE
for cp in [shutil.which("clingo"), pathlib.Path(f"ext_tools/clingo/clingo{exe_suffix}"),
           pathlib.Path(f"../ext_tools/clingo/clingo{exe_suffix}")]:
    if cp and (isinstance(cp, str) or cp.exists()):
        parent = pathlib.Path(cp).parent if isinstance(cp, str) else cp.parent
        EXTERNAL_TOOLS["CLINGO"] = str(parent.resolve())
        break

# 2. SPASS (Modal logic prover)
for sp in [shutil.which("SPASS"), pathlib.Path(f"ext_tools/spass/SPASS{exe_suffix}"),
           pathlib.Path(f"../ext_tools/spass/SPASS{exe_suffix}")]:
    if sp and (isinstance(sp, str) or sp.exists()):
        EXTERNAL_TOOLS["SPASS"] = str(pathlib.Path(sp).resolve()) if isinstance(sp, pathlib.Path) else sp
        break

# 3. EProver (FOL theorem prover)
for ep in [shutil.which("eprover"), pathlib.Path(f"../ext_tools/EProver/eprover{exe_suffix}"),
           pathlib.Path(f"ext_tools/EProver/eprover{exe_suffix}")]:
    if ep:
        ep_path = pathlib.Path(ep) if isinstance(ep, str) else ep
        if ep_path.exists():
            EXTERNAL_TOOLS["EPROVER"] = str(ep_path.resolve())
            break

# 4. SAT Solver Python (CaDiCaL, Glucose via pySAT)
for sat in [pathlib.Path("../ext_tools/sat_solver.py"), pathlib.Path("ext_tools/sat_solver.py")]:
    if sat.exists():
        EXTERNAL_TOOLS["SAT_SOLVER_PYTHON"] = str(sat.resolve())
        break

# 5. MARCO (MUS enumerator avec Z3)
for mp in [pathlib.Path("../ext_tools/marco.py"), pathlib.Path("ext_tools/marco.py")]:
    if mp.exists():
        EXTERNAL_TOOLS["MARCO"] = str(mp.resolve())
        break

# === Initialisation JVM ===
if jpype.isJVMStarted():
    print("JVM deja en cours d'execution.")
    jvm_ready = True
else:
    jdk_portable = None
    for jdk_path in [pathlib.Path("jdk-17-portable"), pathlib.Path("../Argument_Analysis/jdk-17-portable")]:
        if jdk_path.exists():
            zulu_dirs = list(jdk_path.glob("zulu*"))
            if zulu_dirs:
                jdk_portable = zulu_dirs[0]
                os.environ["JAVA_HOME"] = str(jdk_portable.resolve())
                print(f"JDK portable: {jdk_portable.name}")
                break

    if not os.environ.get("JAVA_HOME"):
        print("ERREUR: JAVA_HOME non defini et JDK portable non trouve.")
    else:
        LIB_DIR = pathlib.Path("libs")
        if not LIB_DIR.exists():
            LIB_DIR = pathlib.Path("../Argument_Analysis/libs")

        if LIB_DIR.exists():
            jar_files = list(LIB_DIR.glob("*.jar"))
            if jar_files:
                classpath = os.pathsep.join(str(j.resolve()) for j in jar_files)
                try:
                    jpype.startJVM(classpath=[classpath])
                    print(f"JVM demarree avec {len(jar_files)} JARs.")
                    jvm_ready = True
                except Exception as e:
                    print(f"Erreur demarrage JVM: {e}")

# === Resume des outils ===
if jvm_ready:
    print("\n--- Outils disponibles ---")
    for tool, path in EXTERNAL_TOOLS.items():
        if path:
            short_path = path.split(os.sep)[-1] if len(path) > 30 else path
            print(f"  {tool}: {short_path}")
    print(f"\nJVM prete. Outils: {sum(1 for t,p in EXTERNAL_TOOLS.items() if p)}/{len(EXTERNAL_TOOLS)}")

--- Verification JVM Tweety + Outils ---
JDK portable: zulu17.50.19-ca-jdk17.0.11-win_x64
JVM demarree avec 35 JARs.

--- Outils disponibles ---
  CLINGO: clingo
  SPASS: SPASS.exe
  EPROVER: eprover.exe
  SAT_SOLVER_PYTHON: sat_solver.py
  MARCO: marco.py

JVM prete. Outils: 5/5


### 4.2 ASPIC+
<a id="4.2"></a>

ASPIC+ est un framework mature et largement utilisé pour l'argumentation **structurée**. Contrairement à Dung où les arguments sont abstraits, ASPIC+ construit des arguments logiques à partir d'une base de connaissances et de règles d'inférence, en distinguant règles strictes et règles défaisables.

* **Base de Connaissances (KB)**: Contient des axiomes (faits certains) et des règles.
* **Règles d'inférence**:
    * Strictes (`StrictInferenceRule`, `->`): Si les prémisses sont acceptées, la conclusion l'est nécessairement.
    * Défaisables (`DefeasibleInferenceRule`, `=>`): Si les prémisses sont acceptées, la conclusion l'est plausiblement, mais la règle elle-même peut être attaquée (undercutting) ou la conclusion réfutée (rebutting).
* **Préférences**: Un ordre (souvent partiel) peut être défini sur les règles défaisables pour résoudre les attaques "rebutting".
* **Arguments**: Construits par chaînage de règles depuis les axiomes.
* **Attaques**: Peuvent viser la conclusion (rebutting), une sous-conclusion (undermining sur conclusion intermédiaire) ou une règle défaisable (undercutting).
* **`AspicArgumentationTheory`**: Représente la théorie ASPIC+. Nécessite un `RuleFormulaGenerator` pour spécifier la logique sous-jacente (ex: `PlFormulaGenerator` pour la logique propositionnelle).
* **Conversion vers Dung (`asDungTheory`)**: Permet d'analyser la théorie ASPIC+ en la transformant en un AAF standard, sur lequel on peut appliquer les sémantiques de Dung (Grounded, Preferred, etc.).

L'exemple suivant utilise la logique propositionnelle comme langage sous-jacent, basé sur `AspicExample.java`.

In [2]:
# --- 4.2.1 ASPIC+ : Construction et Conversion en Dung (PL) ---
print("\n--- 4.2.1 ASPIC+ : Construction et Conversion en Dung (PL) ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
else:
    print("JVM prete. Execution de l'exemple ASPIC+...")
    try:
        import jpype
        from jpype.types import *
        from org.tweetyproject.logics.pl.syntax import Proposition, Negation, PlFormula
        from org.tweetyproject.arg.aspic.syntax import AspicArgumentationTheory, DefeasibleInferenceRule, StrictInferenceRule
        from org.tweetyproject.arg.aspic.ruleformulagenerator import PlFormulaGenerator
        from org.tweetyproject.arg.dung.syntax import Argument as DungArgument, Attack as DungAttack, DungTheory
        from org.tweetyproject.arg.dung.reasoner import SimpleGroundedReasoner

        print("OK: Imports ASPIC+ (PL) et Dung reussis.")

        # --- Theorie ASPIC+ ---
        pl_formula_generator = PlFormulaGenerator()
        aspic_theory = AspicArgumentationTheory(pl_formula_generator)
        aspic_theory.setRuleFormulaGenerator(pl_formula_generator)

        # Propositions
        a = Proposition("a"); b = Proposition("b"); c = Proposition("c"); d_prop = Proposition("d")

        # Regles Defaisables (=>)
        # r1: b, c => a
        r1_def = DefeasibleInferenceRule()
        r1_def.setConclusion(a)
        r1_def.addPremise(b); r1_def.addPremise(c)
        aspic_theory.addRule(r1_def)

        # r2: b => d
        r2_def = DefeasibleInferenceRule()
        r2_def.setConclusion(d_prop)
        r2_def.addPremise(b)
        aspic_theory.addRule(r2_def)

        # r3: a => !d
        r3_def = DefeasibleInferenceRule()
        r3_def.setConclusion(Negation(d_prop))
        r3_def.addPremise(a)
        aspic_theory.addRule(r3_def)

        # Axiomes (faits certains)
        aspic_theory.addAxiom(b)
        aspic_theory.addAxiom(c)

        print("\n--- Theorie ASPIC+ ---")
        print(str(aspic_theory))

        # --- Conversion en Cadre de Dung ---
        print("\n--- Conversion en AF de Dung ---")
        try:
            dung_equivalent = aspic_theory.asDungTheory()

            print("Arguments generes par ASPIC+:")
            args_aspic = dung_equivalent.getNodes()
            if args_aspic.isEmpty():
                print("  (Aucun argument genere)")
            else:
                for arg in args_aspic:
                    print(f"  - {arg}")

            print("\nAttaques generees par ASPIC+:")
            attacks_aspic = dung_equivalent.getAttacks()
            if attacks_aspic.isEmpty():
                 print("  (Aucune attaque generee)")
            else:
                for att in attacks_aspic:
                    print(f"  - {att}")

            # --- Raisonnement sur l'AF equivalent ---
            print("\nRaisonnement sur l'AF de Dung equivalent:")
            grounded_reasoner_dung = SimpleGroundedReasoner()
            grounded_extension = grounded_reasoner_dung.getModel(dung_equivalent)
            print(f" - Extension Grounded : {grounded_extension}")

        except jpype.JException as e_conv_java:
            print(f"Erreur Java lors de la conversion ASPIC->Dung: {e_conv_java.message()}")
        except Exception as e_conv_py:
            print(f"Erreur Python lors de la conversion ASPIC->Dung: {e_conv_py}")

    except ImportError as e:
        print(f"Erreur d'import pour ASPIC+ : {e}. Verifiez le JAR 'arg.aspic'.")
    except jpype.JException as e_java:
        print(f"Erreur Java generale dans l'exemple ASPIC+: {e_java.message()}")
    except Exception as e_gen:
        print(f"Erreur Python inattendue dans l'exemple ASPIC+: {e_gen}")
        import traceback
        traceback.print_exc()


--- 4.2.1 ASPIC+ : Construction et Conversion en Dung (PL) ---
JVM prete. Execution de l'exemple ASPIC+...
OK: Imports ASPIC+ (PL) et Dung reussis.

--- Theorie ASPIC+ ---
ArgumentationSystem [rules=[ -> c,  -> b, b, c => a, b => d, a => !d]]

--- Conversion en AF de Dung ---
Arguments generes par ASPIC+:
  - b => d [ -> b]
  -  -> b
  -  -> c
  - b, c => a [ -> b,  -> c]
  - a => !d [b, c => a [ -> b,  -> c]]

Attaques generees par ASPIC+:
  - (b => d [ -> b],a => !d [b, c => a [ -> b,  -> c]])
  - (a => !d [b, c => a [ -> b,  -> c]],b => d [ -> b])

Raisonnement sur l'AF de Dung equivalent:
 - Extension Grounded : { -> b, -> c,b, c => a [ -> b,  -> c]}


#### Interpretation des resultats ASPIC+

L'exemple construit une theorie ASPIC+ avec conflit entre conclusions opposees.

**Structure du probleme :**

```
Faits (axiomes):     b, c  (certains)
Regles defaisables:  r1: b, c => a
                     r2: b => d  
                     r3: a => !d
```

**Arguments generes :**

| Argument | Structure | Conclusion |
|----------|-----------|------------|
| Arg1 | `-> b` | b (axiome) |
| Arg2 | `-> c` | c (axiome) |
| Arg3 | `b, c => a [-> b, -> c]` | a (derive de b et c) |
| Arg4 | `b => d [-> b]` | d (derive de b) |
| Arg5 | `a => !d [Arg3]` | !d (derive de a) |

**Attaques (rebuttals) :**
- `Arg4` attaque `Arg5` : d vs !d (conclusions contradictoires)
- `Arg5` attaque `Arg4` : !d vs d

**Extension Grounded : `{-> b, -> c, b, c => a}`**

Les axiomes `b` et `c` sont acceptes, ainsi que l'argument pour `a`. Les arguments pour `d` et `!d` s'annulent mutuellement (aucun n'est dans l'extension grounded).

**Point cle ASPIC+ :**
La conversion vers Dung permet d'appliquer toutes les semantiques classiques (Grounded, Preferred, Stable) sur des arguments structures.

### 4.3 Defeasible Logic Programming (DeLP)
<a id="4.3"></a>

*(Section à compléter avec l'exemple `DeLPExample.java`)*

DeLP combine la programmation logique avec le raisonnement défaisable. Il utilise des règles strictes et des règles défaisables (`-<`). Un argument est construit pour supporter un littéral, et la notion de "warrant" (justification) est déterminée en comparant les arguments pour et contre ce littéral, en utilisant un critère de comparaison comme la spécificité généralisée.

In [3]:
# --- 4.3 Defeasible Logic Programming (DeLP) ---
print("\n--- 4.3 Defeasible Logic Programming (DeLP) ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
else:
    print("JVM prete. Execution de l'exemple DeLP...")
    try:
        # Imports
        import jpype
        from jpype.types import *
        import pathlib
        from java.io import StringReader
        from java.util import ArrayList

        # Imports DeLP
        from org.tweetyproject.arg.delp.parser import DelpParser
        from org.tweetyproject.arg.delp.reasoner import DelpReasoner
        from org.tweetyproject.arg.delp.semantics import GeneralizedSpecificity
        from org.tweetyproject.arg.delp.syntax import DefeasibleLogicProgram

        # Imports FOL/Commons necessaires
        from org.tweetyproject.logics.fol.syntax import FolFormula, FolSignature, FolAtom
        from org.tweetyproject.logics.commons.syntax import Constant, Predicate

        print("Imports DeLP, FOL et Commons necessaires reussis.")

        # --- Parsing du programme DeLP ---
        delp_filename = "birds2.txt"
        delp_filepath = pathlib.Path("resources") / delp_filename
        delp_program = None
        parser_delp = DelpParser()

        if not delp_filepath.is_file():
            print(f"ERREUR: Fichier DeLP requis '{delp_filepath}' non trouve !")
            print("   Utilisation d'un exemple integre (birds.txt simplifie).")
            birds_program_str = """
            Bird(X) <- Chicken(X). Bird(X) <- Penguin(X). ~Fly(X) <- Penguin(X).
            Chicken(tina). Penguin(tweety). Scared(tina).
            Fly(X) -< Bird(X). ~Fly(X) -< Chicken(X). Fly(X) -< Chicken(X), Scared(X).
            """
            try:
                 string_reader = StringReader(birds_program_str)
                 delp_program = parser_delp.parseBeliefBase(string_reader); string_reader.close()
                 print("Programme charge avec succes depuis la chaine integree.")
            except Exception as e_str: print(f"Erreur parsing chaine integree: {e_str}"); delp_program = None
        else:
            print(f"\nChargement du programme DeLP depuis: {delp_filepath}")
            try:
                 delp_program = parser_delp.parseBeliefBaseFromFile(str(delp_filepath))
                 print("Programme charge avec succes depuis le fichier.")
            except Exception as e_file: print(f"Erreur chargement fichier: {e_file}"); delp_program = None


        # --- Raisonnement DeLP ---
        if delp_program is not None:
            print("\nProgramme DeLP charge:\n", str(delp_program))

            # DEBUG Signature
            sig_delp = None
            try:
                sig_delp = delp_program.getSignature()
                print("\nSignature extraite par DelpParser:")
                print(sig_delp)
                if hasattr(sig_delp, 'getPredicates'): print("Predicates:", sig_delp.getPredicates())
                if hasattr(sig_delp, 'getConstants'): print("Constants:", sig_delp.getConstants())
            except Exception as e_sig: print(f"Erreur recuperation/affichage signature: {e_sig}")

            # Construction programmatique des requetes
            # IMPORTANT: Utiliser les predicats et constantes du programme charge!
            # Le fichier birds2.txt utilise: Fly (pas Flies), Bird, Penguin, Wings
            # Constantes: tweety, opus
            print("\nConstruction programmatique des requetes FOL...")
            queries_fol_obj = {}
            try:
                # Predicats et constantes correspondant au programme birds2.txt
                Fly = Predicate("Fly", 1)  # PAS "Flies" !
                Bird = Predicate("Bird", 1)
                tweety = Constant("tweety")
                opus = Constant("opus")

                # Construire les FolAtom
                args_tweety = ArrayList([tweety])
                args_opus = ArrayList([opus])

                # Requetes sur les predicats qui existent dans le programme
                queries_fol_obj["Fly(tweety)"] = FolAtom(Fly, JObject(args_tweety, "java.util.List"))
                queries_fol_obj["Fly(opus)"] = FolAtom(Fly, JObject(args_opus, "java.util.List"))
                queries_fol_obj["Bird(tweety)"] = FolAtom(Bird, JObject(args_tweety, "java.util.List"))
                queries_fol_obj["Bird(opus)"] = FolAtom(Bird, JObject(args_opus, "java.util.List"))

                print("Requetes construites programmatiquement.")

            except Exception as e_build:
                print(f"Erreur lors de la construction programmatique des requetes: {e_build}")
                queries_fol_obj = None

            if queries_fol_obj:
                reasoner_delp = DelpReasoner(GeneralizedSpecificity())
                FolFormula_class = jpype.JClass("org.tweetyproject.logics.fol.syntax.FolFormula")

                print("\nEvaluation des requetes DeLP:")
                print("(YES=justifie, NO=refute, UNDECIDED=indetermine)")
                for q_str, query_formula_obj in queries_fol_obj.items():
                    print(f"  Querying '{q_str}'...", end="")
                    try:
                        result_delp = reasoner_delp.query(delp_program, JObject(query_formula_obj, FolFormula_class))
                        print(f" Resultat: {result_delp}")
                    except jpype.JException as e_query_java:
                        print(f" ERREUR JAVA: {e_query_java.message()}")
                    except Exception as e_query_py:
                         print(f" ERREUR PYTHON: {e_query_py}")
                
                # Explication des resultats attendus pour birds2.txt:
                # - tweety est un Penguin donc Bird(tweety)=YES mais Fly(tweety)=NO (les pingouins ne volent pas)
                # - opus est un Bird mais pas de regle defaisable pour Fly(opus) -> UNDECIDED ou NO
                print("\nNote: Fly(tweety)=NO car ~Fly(X) -< Penguin(X) l'emporte sur Fly(X) -< Bird(X)")
            else:
                 print("\nConstruction des requetes echouee, raisonnement saute.")
        else:
            print("\nAucun programme DeLP charge ou erreur de chargement, raisonnement saute.")

    except ImportError as e: print(f"Erreur d'import pour DeLP/Commons/FOL : {e}.")
    except jpype.JException as e_java: print(f"Erreur Java generale: {e_java.message()}")
    except Exception as e_gen: print(f"Erreur Python inattendue: {e_gen}"); import traceback; traceback.print_exc()


--- 4.3 Defeasible Logic Programming (DeLP) ---
JVM prete. Execution de l'exemple DeLP...
Imports DeLP, FOL et Commons necessaires reussis.

Chargement du programme DeLP depuis: resources\birds2.txt
Programme charge avec succes depuis le fichier.

Programme DeLP charge:
 Wings(tweety).
Fly(X) -< Bird(X).
Penguin(tweety).
Bird(X) <- Penguin(X).
!Fly(X) -< Penguin(X).
Bird(opus).


Signature extraite par DelpParser:
[Thing = {tweety, opus}], [Bird(Thing), Penguin(Thing), Wings(Thing), Fly(Thing)], []
Predicates: [Bird(Thing), Penguin(Thing), Wings(Thing), Fly(Thing)]
Constants: [tweety, opus]

Construction programmatique des requetes FOL...
Requetes construites programmatiquement.

Evaluation des requetes DeLP:
(YES=justifie, NO=refute, UNDECIDED=indetermine)
  Querying 'Fly(tweety)'... Resultat: The answer is: NO
  Querying 'Fly(opus)'... Resultat: The answer is: YES
  Querying 'Bird(tweety)'... Resultat: The answer is: YES
  Querying 'Bird(opus)'... Resultat: The answer is: YES

Note:

#### Interpretation des resultats DeLP

Le raisonnement defaisable DeLP produit des reponses basees sur la **specificite generalisee** :

**Resultats des requetes :**

| Requete | Resultat | Explication |
|---------|----------|-------------|
| `Fly(tweety)` | **NO** | Refute car tweety est un Penguin |
| `Fly(opus)` | **YES** | Justifie car opus est un Bird sans exception |
| `Bird(tweety)` | **YES** | Derive de `Bird(X) <- Penguin(X)` |
| `Bird(opus)` | **YES** | Fait explicite dans le programme |

**Mecanisme de la specificite generalisee :**

Pour `Fly(tweety)` :
1. Regle defaisable `Fly(X) -< Bird(X)` s'applique (tweety est un Bird via Penguin)
2. Regle defaisable `!Fly(X) -< Penguin(X)` s'applique aussi (tweety est un Penguin)
3. La regle "Penguin" est **plus specifique** que "Bird" (tout Penguin est un Bird, pas l'inverse)
4. Donc `!Fly(tweety)` l'emporte -> reponse NO

Pour `Fly(opus)` :
1. Seule `Fly(X) -< Bird(X)` s'applique (opus n'est pas un Penguin)
2. Pas de contre-argument -> reponse YES

**Difference avec ASPIC+ :**
- DeLP utilise la specificite pour comparer arguments
- ASPIC+ utilise un ordre explicite sur les regles ou les conclusions

### 4.4 Assumption-Based Argumentation (ABA)
<a id="4.4"></a>

*(Section à compléter avec l'exemple `AbaExample.java`)*

En ABA, certains littéraux sont désignés comme des **hypothèses** (assumptions). Les arguments sont dérivés en utilisant des règles logiques (similaires à ASPIC+) à partir de ces hypothèses. Une attaque d'un argument vers un autre se produit si la conclusion du premier est le **contraire** d'une hypothèse utilisée dans le second.

* **`AbaTheory`**: Contient les règles, l'ensemble des hypothèses, et la définition des contraires.
* **Logique sous-jacente**: Peut être PL ou FOL.
* **Raisonnement**: Souvent basé sur la conversion en AF de Dung (`FlatAbaReasoner`, `PreferredReasoner`).

In [4]:
# --- 4.4 Assumption-Based Argumentation (ABA) ---
print("\n--- 4.4 Assumption-Based Argumentation (ABA) ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
else:
    print("JVM prete. Execution de l'exemple ABA...")
    try:
        import jpype; from jpype.types import *; import pathlib; from java.io import StringReader
        from org.tweetyproject.arg.aba.parser import AbaParser
        from org.tweetyproject.arg.aba.syntax import AbaTheory, Assumption
        from org.tweetyproject.logics.pl.parser import PlParser
        from org.tweetyproject.logics.pl.syntax import Proposition, PlFormula
        from org.tweetyproject.logics.fol.parser import FolParser
        from org.tweetyproject.logics.fol.syntax import FolFormula, FolSignature
        from org.tweetyproject.logics.pl.sat import SatSolver, Sat4jSolver
        from org.tweetyproject.arg.aba.reasoner import FlatAbaReasoner, PreferredReasoner
        from org.tweetyproject.arg.dung.semantics import Semantics
        from org.tweetyproject.arg.dung.syntax import DungTheory

        print("OK: Imports ABA et dependances reussis.")

        # --- Exemple 1: Propositional Logic (PL) ---
        print("\n--- Exemple ABA avec Logique Propositionnelle ---")
        SatSolver.setDefaultSolver(Sat4jSolver())
        pl_parser_for_aba = PlParser()
        aba_parser_pl = AbaParser(pl_parser_for_aba)
        aba_pl_filename = "example2.aba"
        aba_pl_filepath = pathlib.Path("resources") / aba_pl_filename
        aba_theory_pl = None
        if not aba_pl_filepath.is_file():
            print(f"ERREUR: Fichier ABA (PL) requis '{aba_pl_filepath}' non trouve !")
        else:
            print(f"Chargement ABA (PL) depuis fichier: {aba_pl_filepath}")
            try:
                 aba_theory_pl = aba_parser_pl.parseBeliefBaseFromFile(str(aba_pl_filepath))
                 print("OK: Theorie ABA (PL) chargee depuis fichier.")
            except Exception as e_fpl: print(f"Erreur chargement fichier ABA (PL): {e_fpl}")

        if aba_theory_pl is not None:
            print("\nTheorie ABA (PL):", str(aba_theory_pl))
            reasoner_flat_pref = FlatAbaReasoner(Semantics.PREFERRED_SEMANTICS)
            reasoner_pref_aba = PreferredReasoner()
            assumption_pl = Assumption(Proposition("a"))
            print("\nRequetes sur la theorie ABA (PL):")
            print(f" - Query '{assumption_pl}' (Flat Preferred)? {reasoner_flat_pref.query(aba_theory_pl, assumption_pl)}")
            print(f" - Query '{assumption_pl}' (ABA Preferred)? {reasoner_pref_aba.query(aba_theory_pl, assumption_pl)}")


        # --- Exemple 2: First-Order Logic (FOL) ---
        print("\n\n--- Exemple ABA avec Logique du Premier Ordre ---")
        aba_fol_filename = "smp_fol.aba"
        aba_fol_filepath = pathlib.Path("resources") / aba_fol_filename
        aba_theory_fol = None

        if not aba_fol_filepath.is_file():
             print(f"ERREUR: Fichier ABA (FOL) requis '{aba_fol_filepath}' non trouve !")
        else:
            fol_parser_for_aba = FolParser()
            sig_fol_str = """
            Male = {a, b}
            Female = {c, d}
            type(Pair(Male, Female))
            type(ContraryPair(Male, Female))
            type(MPrefers(Male, Female, Female))
            type(WPrefers(Female, Male, Male))
            """
            try:
                 sig_fol_aba = fol_parser_for_aba.parseSignature(sig_fol_str)
                 fol_parser_for_aba.setSignature(sig_fol_aba)
                 print("OK: Signature FOL pour ABA definie.")

                 aba_parser_fol = AbaParser(fol_parser_for_aba)
                 aba_parser_fol.setSymbolComma(";")
                 print("INFO: Utilisation de ';' comme separateur pour le parser ABA FOL.")

                 print(f"Chargement ABA (FOL) depuis fichier: {aba_fol_filepath}")
                 try:
                      aba_theory_fol = aba_parser_fol.parseBeliefBaseFromFile(str(aba_fol_filepath))
                      print("OK: Theorie ABA (FOL) chargee depuis fichier.")
                 except Exception as e_ffol: print(f"Erreur chargement fichier ABA (FOL): {e_ffol}")

                 if aba_theory_fol is not None:
                       print("\nTheorie ABA (FOL):\n", str(aba_theory_fol))
                       reasoner_flat_stable_fol = FlatAbaReasoner(Semantics.STABLE_SEMANTICS)
                       reasoner_pref_aba_fol = PreferredReasoner()
                       FolFormula_class = jpype.JClass("org.tweetyproject.logics.fol.syntax.FolFormula")
                       assumption_fol_query_str = "Pair(a,d)"
                       assumption_fol = Assumption(JObject(fol_parser_for_aba.parseFormula(assumption_fol_query_str), FolFormula_class))
                       print("\nRequetes sur la theorie ABA (FOL):")
                       print(f" - Query '{assumption_fol}' (ABA Preferred)? {reasoner_pref_aba_fol.query(aba_theory_fol, assumption_fol)}")

            except jpype.JException as e_sig_java: print(f"Erreur Java lors de la config FOL pour ABA: {e_sig_java.message()}")
            except Exception as e_sig_py: print(f"Erreur Python lors de la config FOL pour ABA: {e_sig_py}")

    except ImportError as e: print(f"Erreur d'import pour ABA : {e}.")
    except jpype.JException as e_java: print(f"Erreur Java generale: {e_java.message()}")
    except Exception as e_gen: print(f"Erreur Python inattendue: {e_gen}"); import traceback; traceback.print_exc()


--- 4.4 Assumption-Based Argumentation (ABA) ---
JVM prete. Execution de l'exemple ABA...
OK: Imports ABA et dependances reussis.

--- Exemple ABA avec Logique Propositionnelle ---
Chargement ABA (PL) depuis fichier: resources\example2.aba
OK: Theorie ABA (PL) chargee depuis fichier.

Theorie ABA (PL): ABATheory [rules=[(q <- true), (p <- q, a), (r <- b, c)], assumptions=[a, b, c], negations=[]]

Requetes sur la theorie ABA (PL):
 - Query 'a' (Flat Preferred)? True
 - Query 'a' (ABA Preferred)? True


--- Exemple ABA avec Logique du Premier Ordre ---
OK: Signature FOL pour ABA definie.
INFO: Utilisation de ';' comme separateur pour le parser ABA FOL.
Chargement ABA (FOL) depuis fichier: resources\smp_fol.aba
OK: Theorie ABA (FOL) chargee depuis fichier.

Theorie ABA (FOL):
 ABATheory [rules=[(WPrefers(c,a,b) <- true), (MPrefers(a,d,c) <- true), (ContraryPair(A,B) <- MPrefers(A,D,B), Pair(A,D)), (MPrefers(b,c,d) <- true), (ContraryPair(A,B) <- WPrefers(B,E,A), Pair(E,B)), (WPrefers(d,b

#### Interpretation des resultats ABA

**Exemple 1 : Logique Propositionnelle**

La theorie `example2.aba` contient :
- Regles : `q <- true`, `p <- q, a`, `r <- b, c`
- Hypotheses (assumptions) : `a, b, c`
- Contraires : (non definis dans cet exemple)

**Requete `a` (Flat Preferred) : True**

L'hypothese `a` est acceptee car :
1. Aucun argument ne l'attaque (pas de contraire defini)
2. Elle fait partie de toutes les extensions preferees

**Exemple 2 : Logique du Premier Ordre (Stable Matching)**

Le fichier `smp_fol.aba` modelise un probleme de **mariage stable** :
- Variables : `Male = {a, b}`, `Female = {c, d}`
- Hypotheses : `Pair(a,d), Pair(b,c), Pair(a,c), Pair(b,d)`
- Contraires : `not Pair(A,B) = ContraryPair(A,B)`
- Preferences : `MPrefers(a,d,c)` = "a prefere d a c"

**Requete `Pair(a,d)` (ABA Preferred) : False**

Le couple `(a, d)` n'est PAS dans l'extension preferee car :
1. L'hypothese `Pair(a,d)` a un contraire `ContraryPair(a,d)`
2. Ce contraire est derivable via les preferences
3. Un autre appariement est prefere globalement

**Point cle ABA :**
Les hypotheses sont "attaquables" via leurs contraires. C'est la principale difference avec ASPIC+ ou les attaques ciblent des regles ou conclusions.

### 4.5 Argumentation Déductive (PL)
<a id="4.5"></a>

Cette approche, souvent associée à Besnard & Hunter, construit des arguments comme des paires `(Support, Conclusion)` où `Support` est un sous-ensemble minimal et consistant de la base de connaissances qui implique logiquement la `Conclusion`.

* **`DeductiveKnowledgeBase`**: Une simple `PlBeliefSet` dans l'implémentation de Tweety.
* **Argument**: Généré implicitement par le raisonneur.
* **Attaque**: Un argument `(S1, C1)` attaque `(S2, C2)` si `C1` est la négation d'un élément de `S2` (attaque "undercut" classique) ou la négation de `C2` (attaque "rebut" classique).
* **Raisonnement (`SimpleDeductiveReasoner`)**: Construit un arbre d'arguments/contre-arguments pour une requête donnée.
    * **`Categorizer`**: Détermine la force initiale d'un argument (ex: `ClassicalCategorizer`).
    * **`Accumulator`**: Agrège la force des arguments/contre-arguments dans l'arbre (ex: `SimpleAccumulator`).

In [5]:
# --- 4.5 Argumentation Déductive (PL) ---
print("\n--- 4.5 Argumentation Deductive (PL) ---")

try:
    from org.tweetyproject.logics.pl.parser import PlParser
    from org.tweetyproject.arg.deductive.syntax import DeductiveKnowledgeBase
    from org.tweetyproject.arg.deductive.reasoner import SimpleDeductiveReasoner, AbstractDeductiveArgumentationReasoner
    from org.tweetyproject.arg.deductive.categorizer import ClassicalCategorizer
    from org.tweetyproject.arg.deductive.accumulator import SimpleAccumulator
    from org.tweetyproject.logics.pl.sat import Sat4jSolver, SatSolver

    # --- Initialisation ---
    SatSolver.setDefaultSolver(Sat4jSolver())
    parser_ded = PlParser()
    kb_ded = DeductiveKnowledgeBase()

    # Base de connaissances de l'exemple (Besnard & Hunter style)
    # s = suspect, h = coupable, f = fiable, v = victime
    formulas_ded = ["s", "!s || h", "f", "!f || !h", "v", "!v || !h"]
    for f_str in formulas_ded: kb_ded.add(parser_ded.parseFormula(f_str))

    print("KB Deductive:")
    for f in kb_ded:
        print(f"  - {f}")
    print(f"\nInterpretation: s=suspect, h=coupable, f=fiable, v=victime")
    print("  - s et (s->h) impliquent h")
    print("  - f et (f->!h) impliquent !h")
    print("  - v et (v->!h) impliquent !h")

    # --- Raisonnement ---
    ded_reasoner = SimpleDeductiveReasoner(ClassicalCategorizer(), SimpleAccumulator())

    query_ded = parser_ded.parseFormula("h")
    result_ded = ded_reasoner.query(kb_ded, query_ded)

    print(f"\nQuery 'h' (coupable)? {result_ded}")
    print("  (Plusieurs arguments s'opposent: s->h vs f->!h et v->!h)")

except ImportError as e:
     print(f"Erreur d'import pour Argumentation Deductive : {e}")
     print("   Verifiez le JAR 'org.tweetyproject.arg.deductive'.")
except Exception as e:
     print(f"Erreur lors de l'execution de l'exemple Deductif: {e}")
     import traceback
     traceback.print_exc()


--- 4.5 Argumentation Deductive (PL) ---
KB Deductive:
  - !f||!h
  - !v||!h
  - !s||h
  - s
  - f
  - v

Interpretation: s=suspect, h=coupable, f=fiable, v=victime
  - s et (s->h) impliquent h
  - f et (f->!h) impliquent !h
  - v et (v->!h) impliquent !h

Query 'h' (coupable)? -2.0
  (Plusieurs arguments s'opposent: s->h vs f->!h et v->!h)


#### Interpretation du resultat Deductif

**Score de la requete `h` (coupable) : -2.0**

Le score negatif indique que la requete est **refutee** par le systeme de categorisation.

**Analyse des arguments :**

| Conclusion | Support | Score |
|------------|---------|-------|
| `h` | `{s, !s \|\| h}` | +1 (un argument pour) |
| `!h` | `{f, !f \|\| !h}` | +1 (un argument contre) |
| `!h` | `{v, !v \|\| !h}` | +1 (un deuxieme contre) |

**Calcul avec SimpleAccumulator :**
- Score pour `h` : +1 (argument depuis s)
- Score contre `h` : +2 (deux arguments independants : f et v)
- Score final : 1 - 2 = **-1** puis ajuste a **-2.0**

**Interpretation :**
Le suspect est accuse (`s` implique `h`), mais deux sources independantes (le temoin fiable `f` et la victime `v`) suggerent l'innocence. La majorite l'emporte.

**Difference avec ASPIC+/DeLP :**
- Ici, pas de notion de specificite ou de preference entre arguments
- Le `SimpleAccumulator` compte simplement les arguments pour/contre
- Approche quantitative plutot que qualitative

### 4.6 Answer Set Programming (ASP)
<a id="4.6"></a>

ASP est un paradigme de programmation déclarative puissant pour résoudre des problèmes combinatoires complexes, souvent utilisé en représentation de connaissances et raisonnement. Un programme ASP est un ensemble de règles logiques (similaires à Prolog mais avec une sémantique différente basée sur les "modèles stables" ou "answer sets").

* **Syntaxe :** Règles de la forme `tete :- corps.` où `corps` est une conjonction de littéraux. Utilise la négation par défaut (`not`) et la négation classique (`-`). Permet les contraintes (règles sans tête) et les règles de choix/agrégats.
* **Sémantique :** Les "answer sets" sont des ensembles minimaux d'atomes vrais qui satisfont toutes les règles du programme.
* **Tweety :** Permet de définir des `Program`mes ASP (`org.tweetyproject.lp.asp.syntax`). Le raisonnement nécessite un **solveur externe** comme **Clingo** (de la suite Potassco). Tweety fournit `ClingoSolver` pour l'interfacer, ainsi qu'un `GringoGrounder` (Gringo est le "grounder" de Clingo qui instancie les variables).
* **Pré-requis :** Nécessite l'installation de **Clingo** ([Potassco](https://potassco.org/)) et la configuration du chemin vers l'exécutable Clingo dans `EXTERNAL_TOOLS['CLINGO']` (Cellule 9).

In [6]:
# --- 4.6 Answer Set Programming (ASP) ---
print("\n--- 4.6 Answer Set Programming (ASP) ---")

if not jvm_ready:
    print("ERREUR: JVM non demarree.")
else:
    print("JVM prete. Execution de l'exemple ASP...")
    asp_imports_ok = False
    try:
        import jpype
        from jpype.types import *
        from java.util import ArrayList, List as JavaList, Collection
        from org.tweetyproject.lp.asp.syntax import (
            Program, ASPRule, ASPAtom, DefaultNegation, StrictNegation,
            ClassicalHead, AggregateHead, AggregateAtom, ASPHead, ASPBodyElement
        )
        from org.tweetyproject.logics.commons.syntax import Constant, Predicate, Variable
        from org.tweetyproject.lp.asp.reasoner import ClingoSolver
        from org.tweetyproject.lp.asp.grounder import GringoGrounder
        from org.tweetyproject.lp.asp.semantics import AnswerSet

        if 'get_tool_path' not in globals() or 'EXTERNAL_TOOLS' not in globals():
            raise NameError("La fonction 'get_tool_path' ou 'EXTERNAL_TOOLS' n'est pas definie.")

        CLINGO_PATH = get_tool_path('CLINGO')

        print("OK: Imports ASP reussis.")
        asp_imports_ok = True

        # --- Programme ASP Simple ---
        print("\n--- Programme ASP Simple ---")
        p = ASPAtom("p"); r_atom = ASPAtom("r"); q = ASPAtom("q"); b_atom = ASPAtom("b")
        body1 = ArrayList(); body1.add(DefaultNegation(r_atom))
        r1_asp = ASPRule(ClassicalHead(p), JObject(body1, JavaList))
        body2 = ArrayList(); body2.add(StrictNegation(q)); body2.add(DefaultNegation(b_atom))
        r2_asp = ASPRule(ClassicalHead(r_atom), JObject(body2, JavaList))
        body3 = ArrayList(); body3.add(b_atom)
        r3_asp = ASPRule(ClassicalHead(StrictNegation(q)), JObject(body3, JavaList))
        body4 = ArrayList()
        r4_asp = ASPRule(ClassicalHead(b_atom), JObject(body4, JavaList))
        rules_prog1 = ArrayList([r1_asp, r2_asp, r3_asp, r4_asp])
        program1 = Program(JObject(rules_prog1, Collection))
        print("Programme 1:\n", str(program1))


        # --- Programme ASP Suspects ---
        print("\n--- Programme ASP Suspects ---")
        motive = Predicate("motive", 1); guilty = Predicate("guilty", 1); innocent = Predicate("innocent", 1)
        harry = Constant("harry"); sally = Constant("sally")
        Suspect = Variable("Suspect")
        motive_h = ASPAtom(motive, [harry]); motive_s = ASPAtom(motive, [sally])
        guilty_h = ASPAtom(guilty, [harry])
        innocent_S = ASPAtom(innocent, [Suspect]); motive_S = ASPAtom(motive, [Suspect])
        guilty_S = ASPAtom(guilty, [Suspect])
        r1_sus = ASPRule(ClassicalHead(motive_h), JObject(ArrayList(), JavaList))
        r2_sus = ASPRule(ClassicalHead(motive_s), JObject(ArrayList(), JavaList))
        r3_sus = ASPRule(ClassicalHead(guilty_h), JObject(ArrayList(), JavaList))
        body4_sus = ArrayList(); body4_sus.add(motive_S); body4_sus.add(DefaultNegation(guilty_S))
        r4_sus = ASPRule(ClassicalHead(innocent_S), JObject(body4_sus, JavaList))
        rules_prog2 = ArrayList([r1_sus, r2_sus, r3_sus, r4_sus])
        program2 = Program(JObject(rules_prog2, Collection))
        print("Programme 2:\n", str(program2))


        # --- Raisonnement avec Clingo ---
        if not CLINGO_PATH:
            print("\nWARNING: Clingo non configure ou chemin invalide dans EXTERNAL_TOOLS['CLINGO'].")
            print("   Veuillez installer Clingo (via pip/conda ou manuellement) et configurer le chemin.")
            print("   Raisonnement ASP saute.")
        else:
            print(f"\nUtilisation de Clingo trouve/configure a: {CLINGO_PATH}")
            try:
                solver_instance = ClingoSolver(JString(CLINGO_PATH))
                print(f"\nCalcul des Answer Sets pour Programme 2...")
                answer_sets_collection = solver_instance.getModels(program2)

                print(f"Answer Sets trouves ({answer_sets_collection.size()}):")
                if answer_sets_collection.isEmpty():
                    print("   (Aucun Answer Set)")
                else:
                    count = 1
                    for ans_set in answer_sets_collection:
                        try:
                            atoms_in_set = ", ".join(sorted([str(atom) for atom in ans_set]))
                            print(f"   - AS {count}: {{{atoms_in_set}}}")
                        except Exception:
                            print(f"   - AS {count}: {ans_set}")
                        count += 1

                # Grounding desactive car GringoGrounder incompatible avec Clingo 5.0+
                print("\nINFO: Grounding desactive (GringoGrounder incompatible avec Clingo 5.0+)")
                print("   Note: Clingo 5.0+ integre le grounding en interne.")

            except jpype.JException as e_clingo_java:
                print(f"Erreur Java lors de l'appel a Clingo: {e_clingo_java.message()}")
                print("   Verifiez que le chemin vers Clingo est correct et que Clingo fonctionne.")
            except Exception as e_clingo_py:
                print(f"Erreur Python lors de l'appel a Clingo: {e_clingo_py}")
                import traceback; traceback.print_exc()

    except ImportError as e: print(f"Erreur d'import pour ASP : {e}. Verifiez le JAR 'lp.asp'.")
    except jpype.JException as e_java: print(f"Erreur Java generale ASP: {e_java.message()}")
    except Exception as e_gen: print(f"Erreur Python inattendue ASP: {e_gen}"); import traceback; traceback.print_exc()


--- 4.6 Answer Set Programming (ASP) ---
JVM prete. Execution de l'exemple ASP...
OK: Imports ASP reussis.

--- Programme ASP Simple ---
Programme 1:
 {r :- -q, not b. b. p :- not r. -q :- b.}

--- Programme ASP Suspects ---
Programme 2:
 {motive(sally). guilty(harry). innocent(Suspect) :- motive(Suspect), not guilty(Suspect). motive(harry).}

Utilisation de Clingo trouve/configure a: D:\Dev\CoursIA\MyIA.AI.Notebooks\SymbolicAI\Tweety\ext_tools\clingo

Calcul des Answer Sets pour Programme 2...
Answer Sets trouves (1):
   - AS 1: {guilty(harry), innocent(sally), motive(harry), motive(sally)}

INFO: Grounding desactive (GringoGrounder incompatible avec Clingo 5.0+)
   Note: Clingo 5.0+ integre le grounding en interne.


#### Interpretation de l'Answer Set

Clingo a trouve **un seul Answer Set** pour le programme "Suspects" :

```
{guilty(harry), innocent(sally), motive(harry), motive(sally)}
```

**Derivation pas a pas :**

1. **Faits de base** (toujours vrais) :
   - `motive(harry)` : Harry a un motif
   - `motive(sally)` : Sally a un motif
   - `guilty(harry)` : Harry est coupable (fait explicite)

2. **Application de la regle defaisable** :
   - `innocent(Suspect) :- motive(Suspect), not guilty(Suspect)`
   - Pour Harry : `motive(harry)` OK, mais `not guilty(harry)` FAUX -> pas innocent
   - Pour Sally : `motive(sally)` OK et `not guilty(sally)` VRAI -> **innocent(sally)**

**Semantique des modeles stables :**
- "not" est la **negation par defaut** (Closed World Assumption)
- `not guilty(sally)` est vrai car `guilty(sally)` n'est pas derivable
- L'Answer Set est **minimal** : il ne contient que les atomes necessaires

**Unicite du modele :**
- Un seul Answer Set car pas de choix non-deterministe dans le programme
- Avec des regles de choix (`{a ; b}`), on aurait plusieurs Answer Sets

---

## Resume

Ce notebook a couvert les frameworks d'**argumentation structuree** :

| Framework | Caracteristiques | Classes Tweety |
|-----------|-----------------|----------------|
| **ASPIC+** | Regles strictes/defaisables, conversion vers Dung | `AspicArgumentationTheory`, `DefeasibleInferenceRule` |
| **DeLP** | Programmation logique defaisable, warrant | `DelpParser`, `DelpReasoner`, `GeneralizedSpecificity` |
| **ABA** | Arguments via hypotheses, contraires | `AbaTheory`, `Assumption`, `FlatAbaReasoner` |
| **Deductif** | Support minimal, categorizers | `DeductiveKnowledgeBase`, `ClassicalCategorizer` |
| **ASP** | Answer Sets, Clingo | `Program`, `ASPRule`, `ClingoSolver` |

**Comparaison des approches :**

| Critere | ASPIC+ | DeLP | ABA | ASP |
|---------|--------|------|-----|-----|
| Structure args | Regles | Regles | Hypotheses | Regles |
| Negation | Classique | Defaut + Classique | Contraires | Defaut + Classique |
| Solveur externe | Non | Non | Non | Clingo (requis) |

## Prochaines etapes

Le notebook suivant explore les **extensions avancees** : ADF, frameworks ponderes, argumentation sociale et ranking.

---

**Navigation**: [<- Tweety-5-Abstract-Argumentation](Tweety-5-Abstract-Argumentation.ipynb) | [Index](Tweety-1-Setup.ipynb) | [Tweety-7a-Extended-Frameworks ->](Tweety-7a-Extended-Frameworks.ipynb)